TLDR:  
embed abstracts,  
kmeans clustered by abstracts by semantic meaning,  
we grab tags from each cluster and assign them to each URL,  
we then run k-means again for x amount of runs,  
at the end for each url we add up the amount of times it was assigned each tag and take the top 5 tags. 

next steps for improving tags:  
sort out the meaningless tags manually D:  
adjust num runs and k cluster numbers  
maybe only use the top 75ish most popular tags?  
Right now there are 162 unique tags  

In [4]:
import pandas as pd
from keybert import KeyBERT
from sklearn.cluster import KMeans
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
df = pd.read_csv('cleaned_url_data.csv')

df.head()

d:\anaconda\envs\search_engine\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,URL,HTML,Title,Abstract,Publish_date,Authors,KMeansTags
0,https://deepmind.google/research/publications/...,Automated Discovery of Interpretable Cognitive...,Automated Discovery of Interpretable Cognitive...,A principal goal of computational neuroscience...,6 February 2025,"Pablo Castro Rivadeneira, Kim Stachenfeld, Kev...","['data', 'neural', 'networks', 'features', 'fe..."
1,https://deepmind.google/research/publications/...,Scaling Pre-training to One Hundred Billion Da...,Scaling Pre-training to One Hundred Billion Da...,We provide an empirical investigation of the p...,11 February 2025,"Xiao Wang, Ibrahim Alabdulmohsin, Daniel Salz,...","['vision', 'scale', 'image', 'dense', 'text']"
2,https://deepmind.google/research/publications/...,Delta Variances - Google DeepMind ...,Generate videos in Gemini and Whisk with Veo 2...,Decision makers may suffer from uncertainty in...,20 February 2025,"Simon Schmitt, John Shawe-Taylor, Hado van Has...","['planning', 'inference', 'energy', 'variation..."
3,https://deepmind.google/research/publications/...,Effective Kernel Fuzzing with Learned White-bo...,Veo 2 15 April 2025 Effectiv...,Kernel fuzzers rely heavily on program mutatio...,1 April 2025,"Sishuai Gong, Wang Rui, Deniz Altinbüken, Pedr...","['hash', 'kernel', 'keys', 'snowplow', 'covera..."
4,https://deepmind.google/research/publications/...,TIPS: Text-Image Pretraining with Spatial awar...,Whisk with Veo 2 15 April 2025 ...,While image-text representation learning has b...,10 March 2025,"Kevis-Kokitsi Maninis, Kaifeng Chen, Soham Gho...","['vision', 'scale', 'image', 'dense', 'text']"


This creates semantic embeddings of the abstracts, this will allow us to cluster each paper semantically when using k-means

In [5]:

#  pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

abstract_embeddings = model.encode(df['Abstract'])
print(abstract_embeddings.shape)

(278, 384)


Manual implementation of kemans using hw 3 as a start

In [10]:
import numpy as np

def simple_kmeans(X, k=4, tol=1e-4, max_iter=30):
    n_samples, n_features = X.shape

    # random centroids
    indices = np.random.choice(n_samples, size=k, replace=False)
    centroids = X[indices]

    for _ in range(max_iter):
        # Compute distances and assign clusters
        dists = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)  # (n_samples, k)
        labels = np.argmin(dists, axis=1)

        # Update centroids
        new_centroids = np.array([X[labels == i].mean(axis=0) if np.any(labels == i) else centroids[i] for i in range(k)])
        
        # Check convergence
        shifts = np.linalg.norm(new_centroids - centroids, axis=1)
        if np.all(shifts < tol):
            break
        
        centroids = new_centroids

    # Compute final inertia (sum of squared distances)
    final_dists = np.linalg.norm(X - centroids[labels], axis=1)
    inertia = np.sum(final_dists ** 2)
    
    return centroids, labels, inertia


Trying to find a optimal k to use, using inertia which is the the sum of the squared distances between each point and its cluster's center. 

In [ ]:
inertias = []
k_values = range(1, 100)  
for k in k_values:
    centroids, labels, inertia = simple_kmeans(abstract_embeddings, k=k)
    # kmeans.fit(abstract_embeddings)
    inertias.append(inertia)  

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(k_values, inertias, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Looking for Optimal k')
plt.grid(True)
plt.show()

AttributeError: 'tuple' object has no attribute 'inertia_'

hmm I want to pick a higher than normal k to get more tags, but there isn't a obvious elbow except maybe at like k = 15ish,   
But i want to use a higher k than 15 since I want some more specific tags.  
I will try with k = 50 first, meaning theres around 5 papers per cluster  

Running k means multiple times to get different initial clusters. 

In [ ]:
num_kmeans_runs = 20 
k = 40

labels_all_runs = np.zeros((num_kmeans_runs, len(abstract_embeddings)))  

for i in range(num_kmeans_runs):

    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(abstract_embeddings)
    labels_all_runs[i] = kmeans.labels_  

Was debating over Tfid or bag of words, bag of words is simple to implement from scratch, but other than that is just a worse algorithm for tagging. Tfid assigns weight to words given all the abstracts allowing it to pick out more signifigant words. It also  uses stopword to prevent only catching the most popular words such as: "is, and, to, then,". Learn more about it here: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


In [107]:
vectorizer = TfidfVectorizer(stop_words='english', 
                             max_features=1000,
                             token_pattern=r'(?u)\b[a-zA-Z][a-zA-Z]+\b' )#token pattern is regex for each feature to be as least 2 letters and no numbers

tfidf_matrix = vectorizer.fit_transform(df['Abstract'])
potential_tags = np.array(vectorizer.get_feature_names_out())
potential_tags[:20]

array(['ability', 'able', 'abstract', 'access', 'according', 'accuracy',
       'accurate', 'accurately', 'achieve', 'achieved', 'achieves', 'act',
       'action', 'actions', 'activations', 'active', 'actually', 'adapt',
       'adaptation', 'adapting'], dtype=object)

In [ ]:
from collections import defaultdict, Counter
tag_counts_per_url = defaultdict(Counter)

# run for each set of clusters created by k-means 
for labels in labels_all_runs:
    cluster_tags = {}

    # Extract tags
    for cluster_num in range(k):
        cluster_indexes = np.where(labels == cluster_num)[0]
        if len(cluster_indexes) > 0:
            cluster_tfidf = tfidf_matrix[cluster_indexes].mean(axis=0).A1
        else:
            cluster_tfidf = np.zeros(tfidf_matrix.shape[1])  
        top_indices = np.argsort(cluster_tfidf)[::-1][:5]
        top_indices = top_indices[::-1][:5]
        cluster_tags[cluster_num] = potential_tags[top_indices]

    # Assign tags to each URL
    for i in range(len(labels)):
        for tag in cluster_tags[labels[i]]:
            tag_counts_per_url[i][tag] += 1


Finally we grab the top 5 tags for each URL

In [ ]:
final_tags = {}
for i, counter in tag_counts_per_url.items():
    final_tags[i] = [tag for tag, _ in counter.most_common(5)]  


In [110]:
all_final_tags = set(tag for tags in final_tags.values() for tag in tags)
print('total unique  kmeans tags: ', len(all_final_tags))


total unique  kmeans tags:  162


Now we should probably check the tags against the actual abstracts manually

In [115]:
for i in range(5):
    print('Title: ', df["Title"][i])
    print('URL: ', df["URL"][i])
    print('Tags: ', final_tags[i])
    

Title:  Automated Discovery of Interpretable Cognitive Programs underlying Reward-guided behavior
URL:  https://deepmind.google/research/publications/130468
Tags:  ['data', 'neural', 'networks', 'features', 'feature']
Title:  Scaling Pre-training to One Hundred Billion Data for Vision Language Models
URL:  https://deepmind.google/research/publications/132991
Tags:  ['vision', 'scale', 'image', 'dense', 'text']
Title:  Generate videos in Gemini and Whisk with Veo 2 15 April 2025                   Delta Variances
URL:  https://deepmind.google/research/publications/112791
Tags:  ['planning', 'inference', 'energy', 'variational', 'delta']
Title:  Veo 2 15 April 2025                   Effective Kernel Fuzzing with Learned White-box Test Mutators
URL:  https://deepmind.google/research/publications/127036
Tags:  ['hash', 'kernel', 'keys', 'snowplow', 'coverage']
Title:  Whisk with Veo 2 15 April 2025                   TIPS: Text-Image Pretraining with Spatial awareness
URL:  https://deepmind.

Now did using k-means actually make a difference? Lets check!  
Baseline is what TFID would have done by itself on each abstract no k-means

In [112]:
baseline_tags = {} #No k-means
for i in range(tfidf_matrix.shape[0]):
    row = tfidf_matrix[i].toarray().flatten()
    top_indices = np.argsort(row)[::-1][:5] 
    baseline_tags[i] = list(potential_tags[top_indices])

all_baseline_tags = set(tag for tags in baseline_tags.values() for tag in tags)
print('total unique  baseline tags: ', len(all_baseline_tags))

total unique  baseline tags:  542


We can see that since our total unque tags for kmeans is much lower, this means that k-means was able to generalize the tags/topics better accross the papers.

There is bit a of similarity but the kmeans definitely signifigantly changes the tags generated from each abstract.

In [116]:
overlaps = []
for i in range(len(df)):
    set_kmeans = set(final_tags.get(i, [])) 
    set_baseline = set(baseline_tags.get(i, [])) 

    intersection = set_kmeans & set_baseline
    union = set_kmeans | set_baseline

    if union:  
        jaccard = len(intersection) / len(union)
        overlaps.append(jaccard)

print(f"Jaccards similarity  {np.mean(overlaps):.4f}")

Jaccards similarity  0.2318


Taking a closer look at the different tags:

In [114]:
for i in range(3): 
    print('Title: ', df["Title"][i])
    print("KMeans Tags:", final_tags[i])
    print("Baseline Tags:", baseline_tags[i])
    print("-" * 40)


Title:  Automated Discovery of Interpretable Cognitive Programs underlying Reward-guided behavior
KMeans Tags: ['data', 'neural', 'networks', 'features', 'feature']
Baseline Tags: ['programs', 'cogfunsearch', 'discover', 'behavioral', 'program']
----------------------------------------
Title:  Scaling Pre-training to One Hundred Billion Data for Vision Language Models
KMeans Tags: ['vision', 'scale', 'image', 'dense', 'text']
Baseline Tags: ['scale', 'web', 'diversity', 'gains', 'examples']
----------------------------------------
Title:  Generate videos in Gemini and Whisk with Veo 2 15 April 2025                   Delta Variances
KMeans Tags: ['planning', 'inference', 'energy', 'variational', 'delta']
Baseline Tags: ['delta', 'neural', 'uncertainty', 'networks', 'epistemic']
----------------------------------------


To imrpove tagging: I'm currently seeing some tag words that are a bit meaningless. In what we printed above we see that feature and features were keywords from the first abstract, these tags don't really make sense to me right now. To improve this we can add custom stopwords that prevent our TFID model from considering those words. We would have to manually add those stop words though.  

Other steps we can do to improve tagging is messing around with k for k means, maybe we can select random ks between a range(this might be annmoying to implemnt )? k =40 was kind of arbitually selected. Also we can look into howv many iterations of k means to do. 

Another thing we can do to clean the tags is to count up the numbers of each tag and say we only want the top 75 or so and then remove tags from urls that arent in the top 75

In [122]:

df['KMeansTags'] = df.index.map(lambda i: final_tags.get(i, []))

df.head()

,URL,HTML,Title,Abstract,Publish_date,Authors,KMeansTags
0,https://deepmind.google/research/publications/...,Automated Discovery of Interpretable Cognitive...,Automated Discovery of Interpretable Cognitive...,A principal goal of computational neuroscience...,6 February 2025,"Pablo Castro Rivadeneira, Kim Stachenfeld, Kev...","[data, neural, networks, features, feature]"
1,https://deepmind.google/research/publications/...,Scaling Pre-training to One Hundred Billion Da...,Scaling Pre-training to One Hundred Billion Da...,We provide an empirical investigation of the p...,11 February 2025,"Xiao Wang, Ibrahim Alabdulmohsin, Daniel Salz,...","[vision, scale, image, dense, text]"
2,https://deepmind.google/research/publications/...,Delta Variances - Google DeepMind ...,Generate videos in Gemini and Whisk with Veo 2...,Decision makers may suffer from uncertainty in...,20 February 2025,"Simon Schmitt, John Shawe-Taylor, Hado van Has...","[planning, inference, energy, variational, delta]"
3,https://deepmind.google/research/publications/...,Effective Kernel Fuzzing with Learned White-bo...,Veo 2 15 April 2025 Effectiv...,Kernel fuzzers rely heavily on program mutatio...,1 April 2025,"Sishuai Gong, Wang Rui, Deniz Altinbüken, Pedr...","[hash, kernel, keys, snowplow, coverage]"
4,https://deepmind.google/research/publications/...,TIPS: Text-Image Pretraining with Spatial awar...,Whisk with Veo 2 15 April 2025 ...,While image-text representation learning has b...,10 March 2025,"Kevis-Kokitsi Maninis, Kaifeng Chen, Soham Gho...","[vision, scale, image, dense, text]"


In [123]:
df.to_csv('cleaned_url_data.csv', index=False)


In [120]:
# This feels like cheating but likeee
kw_model = KeyBERT('all-MiniLM-L6-v2')

keywords = kw_model.extract_keywords(df["Abstract"], top_n=5)


In [121]:
for i in range(15):
    print('Tags: ', final_tags[i])
    print('ketbert: ', keywords[i])
    

Tags:  ['data', 'neural', 'networks', 'features', 'feature']
ketbert:  [('behavioral', 0.408), ('reward', 0.3432), ('neural', 0.3415), ('program', 0.3414), ('predictive', 0.3361)]
Tags:  ['vision', 'scale', 'image', 'dense', 'text']
ketbert:  [('multilinguality', 0.4022), ('multimodal', 0.3813), ('captions', 0.3723), ('diversity', 0.3378), ('cultural', 0.3288)]
Tags:  ['planning', 'inference', 'energy', 'variational', 'delta']
ketbert:  [('uncertainty', 0.4599), ('variances', 0.3109), ('networks', 0.2981), ('empirically', 0.2965), ('variance', 0.2913)]
Tags:  ['hash', 'kernel', 'keys', 'snowplow', 'coverage']
ketbert:  [('fuzzing', 0.4505), ('mutation', 0.4053), ('mutations', 0.3719), ('coverage', 0.35), ('kernels', 0.3409)]
Tags:  ['vision', 'scale', 'image', 'dense', 'text']
ketbert:  [('captions', 0.3986), ('deepmind', 0.377), ('text', 0.3234), ('supervised', 0.3084), ('learning', 0.2996)]
Tags:  ['causal', 'time', 'arrow', 'coarse', 'entropy']
ketbert:  [('entropy', 0.5618), ('ther